<a href="https://colab.research.google.com/github/jm-tan-jm/web_scrap_agent_info/blob/main/get_Agent_Info_propertyguru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## required multiple functions as the main url dont have the agent's info but only the agent profile link

In [ ]:
!pip install selenium webdriver_manager

In [ ]:
# function 1 - get the agent info link from the property guru url and required region
## Limitation - do one page only each time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_agent_links(core_url="https://www.propertyguru.com.my/property-agents", region=None, page=None):
    if region is None:
        raise ValueError("Please provide valid values for 'region'")

    options = Options()
    options.headless = True
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)
    driver.minimize_window()

    try:
        if page == 1:
            url = rf"{core_url}/{region}"
        else:
            url = rf"{core_url}/{region}/{page}"

        driver.get(url)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'agent-list'))
        )

        agent_links = driver.find_elements(By.XPATH, "//a[contains(@class, 'agent-contact-button')]")
        all_agent_links = [agent_link.get_attribute('href') for agent_link in agent_links]

        return all_agent_links

    except Exception as e:
        print(f"Error: {e}")
        return []

    finally:
        driver.quit()

In [ ]:
# function 2 - get the agent's name and phone number from the url
## Limitation - can only handle one link each time, need to iterate the function for each link
def get_agent_info(agent_link):
    agent_info = {}

    options = Options()
    options.headless = True
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)
    driver.minimize_window()

    try:
        driver.get(agent_link)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'agent-name-wrapper'))
        )

        agent_name_element = driver.find_element(By.CLASS_NAME, 'agent-name')
        agent_name = agent_name_element.text.strip()

        phone_number_element = driver.find_element(By.CSS_SELECTOR, '.contact-agent-actions .action-desktop')
        phone_number = phone_number_element.get_attribute('phonenumber')

        agent_info = {
            'agent_name': agent_name,
            'phone_number': phone_number
        }

    except Exception as e:
        print(f"Error: {e}")

    finally:
        driver.quit()

    return agent_info

In [ ]:
# to run above two functions and convert the result to Excel
import pandas as pd

# to get multiple agents' profile
all_agent_links = []

# loop for multiple pages
for page_number in range(1, 37):  # Scrapes pages 1 to 36
    # function 1
    agent_links = get_agent_links(core_url="https://www.propertyguru.com.my/property-agents", region="melaka", page=page_number)
    all_agent_links.extend(agent_links)

agent_info_list = []

# to get agent's info from each links above
for agent_link in all_agent_links:
    # function 2
    agent_info = get_agent_info(agent_link)
    agent_info_list.append((agent_info['agent_name'], agent_info['phone_number']))

# Convert the list to a DataFrame
df = pd.DataFrame(agent_info_list, columns=['Agent Name', 'Phone Number'])

print(df)

with open('output.txt', 'w') as f:
  print(df, file=f)